In [27]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# ✅ Nate 뉴스 섹션 URL
section_urls = {
    '최신뉴스': 'https://news.nate.com/recent?mid=n0100',
    '정치': 'https://news.nate.com/recent?mid=n0101',
    '경제': 'https://news.nate.com/recent?mid=n0102',
    '사회': 'https://news.nate.com/recent?mid=n0103',
    '세계': 'https://news.nate.com/recent?mid=n0104',
    'IT/과학': 'https://news.nate.com/recent?mid=n0105'
}

def scrape_nate_news(section_name, limit=5):
    """Nate 뉴스 스크래핑: 제목, 링크, 이미지 출력"""
    if section_name not in section_urls:
        print(f"'{section_name}' 섹션이 존재하지 않습니다.")
        return

    url = section_urls[section_name]
    print(f"\n--- {section_name} 뉴스 ---")
    print(f"URL: {url}")

    headers = {"User-Agent": "Mozilla/5.0"}
    try:
        res = requests.get(url, headers=headers)
        res.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return

    soup = BeautifulSoup(res.text, "html.parser")

    # ✅ 섹션별 HTML 구조 확인 → 뉴스 항목 선택
    news_items = soup.select('div.mduSubjectList')

    if not news_items:
        print("뉴스 기사를 찾을 수 없습니다. HTML 구조 확인 필요.")
        return

    for idx, item in enumerate(news_items[:limit], 1):
        # ✅ 제목
        title_tag = item.select_one('h2.tit') or item.select_one('a.lt1')
        title = title_tag.text.strip() if title_tag else "제목 없음"

        # ✅ 링크
        link_tag = item.select_one('a[href]')
        link = urljoin(url, link_tag['href']) if link_tag else "링크 없음"

        # ✅ 이미지
        img_tag = item.select_one('img')
        img_src = urljoin(url, img_tag['src']) if img_tag and img_tag.has_attr('src') else None

        # ✅ 출력
        print(f"\n{idx}. 기사 제목: {title}")
        print(f"   기사 링크: {link}")
        if img_src:
            print(f"   이미지 경로: {img_src}")
            display(Image(url=img_src, width=150))
        else:
            print("   이미지 없음")

# ✅ 전체 섹션 실행
for section in section_urls.keys():
    scrape_nate_news(section)




--- 최신뉴스 뉴스 ---
URL: https://news.nate.com/recent?mid=n0100

1. 기사 제목: 이민정, 영탁도 감탄한 노래 실력…"과거 성악 전공해" (가오정)
   기사 링크: https://news.nate.com/view/20250724n42645?mid=n0100
   이미지 경로: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/is/2025/07/24/isp20250724000368.800x.0.jpg



2. 기사 제목: '초유의 日투어 취소 논란 끝낸 라쿠텐의 결단' 바르셀로나 선수단 곧 日고베행 비행기 오른다[西매체 속보]
   기사 링크: https://news.nate.com/view/20250724n42602?mid=n0100
   이미지 경로: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/sc/2025/07/24/2025072501001806600252191.jpg



3. 기사 제목: 안성 서운면 돼지농장 불…돈사 1개동 소실·인명피해 없어
   기사 링크: https://news.nate.com/view/20250724n42643?mid=n0100
   이미지 경로: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/na/2025/07/24/7412448_high.jpg



4. 기사 제목: 서동주, '♥4세연하 남편'과 독특 신혼집 인테리어 공개 "문 없는 화장실"
   기사 링크: https://news.nate.com/view/20250724n42642?mid=n0100
   이미지 경로: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/su/2025/07/24/760996_1221618_857.png



5. 기사 제목: '천번의 낮밤' 지나도 못 밝힌 이태원의 그날…기억하려 모였다
   기사 링크: https://news.nate.com/view/20250724n42132?mid=n0100
   이미지 경로: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/hn/2025/07/24/53_17533637178466_20250724504344.jpg


...


--- IT/과학 뉴스 ---
URL: https://news.nate.com/recent?mid=n0105

1. 기사 제목: 위고비도 신세계였는데 이번엔…"한 달 뒤 끝판왕 온다" 들썩
   기사 링크: https://news.nate.com/view/20250724n35340?mid=n0105
   이미지 경로: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/hk/2025/07/24/AA.41214182.4.jpg



2. 기사 제목: 크래프톤, '인수 대박'이 '3447억 소송 대란'으로…M&A 후폭풍 맞나
   기사 링크: https://news.nate.com/view/20250724n42325?mid=n0105
   이미지 경로: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/ee/2025/07/24/20250724224438520587.jpg



3. 기사 제목: 카톡·쿠팡 쓰면 내 정보는 어디까지…정부, 슈퍼앱 개인정보 침해에 '칼' 댔다
   기사 링크: https://news.nate.com/view/20250724n42234?mid=n0105
   이미지 경로: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/ee/2025/07/24/20250724223716106226.jpg



4. 기사 제목: 한컴, 사상 첫 파업…'역대급 실적'인데 한컴 직원들 뿔나게 한 '자회사 역차별' 논란
   기사 링크: https://news.nate.com/view/20250724n42021?mid=n0105
   이미지 경로: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/ee/2025/07/24/20250724222053723807.png



5. 기사 제목: AI 주권의 첫 시험대, '뉴스는 공짜가 아니다' 뉴스저작권의 가치를 묻다
   기사 링크: https://news.nate.com/view/20250724n37188?mid=n0105
   이미지 경로: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/ee/2025/07/24/20250724182312696729.jpg


In [ ]:
import os
import requests
from bs4 import BeautifulSoup

def download_one_episode(title, no, url):
    """
    네이버 웹툰의 특정 회차 이미지를 다운로드하는 함수
    :param title: 웹툰 제목
    :param no: 회차 번호
    :param url: 회차 URL
    """
    save_dir = os.path.join('img', title, str(no))
    os.makedirs(save_dir, exist_ok=True)
    print(f"'{save_dir}' 디렉토리에 이미지를 저장합니다.")

    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Referer': url
        }
        
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        image_area = soup.select_one('div.wt_viewer')
        if not image_area:
            print("웹툰 이미지 영역을 찾을 수 없습니다.")
            return

        img_tags = image_area.find_all('img')
        
        for i, img_tag in enumerate(img_tags):
            if 'src' in img_tag.attrs:
                img_url = img_tag['src']
                
                print(f"{i+1}번째 이미지 다운로드 중...", end='\r')
                img_response = requests.get(img_url, headers=headers)
                img_response.raise_for_status()

                file_name = f"{i+1:03d}.jpg"
                file_path = os.path.join(save_dir, file_name)
                
                with open(file_path, 'wb') as f:
                    f.write(img_response.content)
        
        print(f"\n총 {len(img_tags)}개의 이미지를 성공적으로 다운로드했습니다.")

    except requests.exceptions.RequestException as e:
        print(f"HTTP 요청 오류 발생: {e}")
    except Exception as e:
        print(f"오류 발생: {e}")

webtoon_title = '일렉시드'
episode_no = 341
episode_url = 'https://comic.naver.com/webtoon/detail?titleId=717481&no=341&week=wed'

download_one_episode(webtoon_title, episode_no, episode_url)

result_value = f"download_one_episode('{webtoon_title}', {episode_no}, '{episode_url}')"
print(f"\n{result_value}")

'img\일렉시드\341' 디렉토리에 이미지를 저장합니다.
88번째 이미지 다운로드 중...
총 88개의 이미지를 성공적으로 다운로드했습니다.

download_one_episode('일렉시드', 341, 'https://comic.naver.com/webtoon/detail?titleId=717481&no=341&week=wed')
